# Introdução

Bem-vindos ao nosso workshop da Le Wagon em parceria com o INSPER Dynamics e o INSPER GAS! Hoje, mergulharemos na competição do Kaggle intitulada "Titanic: Machine Learning from Disaster" na qual seremos desafiados a utilizar técnicas de machine learning para prever quais passageiros do trágico naufrágio do RMS Titanic conseguiram sobreviver e quais não tiveram a mesma sorte.

O naufrágio do Titanic em 1912 é um dos eventos mais icônicos da história marítima, e essa competição nos oferece a oportunidade de aplicar conhecimentos de ciência de dados e análise preditiva a um problema real. A bordo deste workshop, você terá a oportunidade de explorar dados, desenvolver modelos de aprendizado de máquina e aprimorar suas habilidades de programação.

Durante este evento, vamos fornecer orientações, dicas e técnicas para ajudá-lo a enfrentar esse desafio e aprimorar suas habilidades em aprendizado de máquina.


link da competição: https://www.kaggle.com/competitions/titanic

No código abaixo, vamos realizar o download dos arquivos usando o drive para facilitar o desenvolvimento do workshop.


In [3]:
import gdown

gdown.download(id='1EGaan07Ix9qmXWRImE-kT-rQRV2clktw')

!unzip -o 'workshop.zip'
!rm 'workshop.zip'


Downloading...
From: https://drive.google.com/uc?id=1EGaan07Ix9qmXWRImE-kT-rQRV2clktw
To: /content/workshop.zip
100%|██████████| 67.8k/67.8k [00:00<00:00, 87.8MB/s]

Archive:  workshop.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: titanic.csv             
  inflating: train.csv               


In [4]:
import numpy as np
import pandas as pd

gender_submission = pd.read_csv("gender_submission.csv")
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Análise dos dados

Para visualizar os dados da competição, clique na guia "Data" no topo da página da competição. Em seguida, role para baixo para encontrar a lista de arquivos.
Existem três arquivos nos dados: (1) train.csv, (2) test.csv e (3) gender_submission.csv.


1 - train.csv

O arquivo train.csv contém os detalhes de um subconjunto dos passageiros a bordo (exatamente 891 passageiros - onde cada passageiro ocupa uma linha diferente na tabela). Para investigar esses dados, clique no nome do arquivo à esquerda da tela. Depois de fazer isso, você poderá visualizar todos os dados na janela.


Os valores na segunda coluna ("Survived") podem ser usados para determinar se cada passageiro sobreviveu ou não:

- Se for "1", o passageiro sobreviveu.
- Se for "0", o passageiro faleceu.

In [5]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Como pode ser visto acima, o primeiro passageiro desse dataset se chamava Mr. Owen Harris Braund e não sobreviveu à tragédia.

2 - test.csv

Usando os padrões que você encontra no arquivo train.csv, você deve prever se os outros 418 passageiros a bordo (no arquivo test.csv) sobreviveram.

Clique em test.csv (à esquerda da tela) para examinar seu conteúdo. Observe que o arquivo test.csv não possui uma coluna "Survived" - essa informação está oculta, e o quão bem você consegue prever esses valores ocultos determinará sua pontuação na competição!

In [6]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S



3 - gender_submission.csv

O arquivo gender_submission.csv é fornecido como um exemplo que mostra como você deve estruturar suas previsões. Ele prevê que todas as passageiras do sexo feminino sobreviveram e que todos os passageiros do sexo masculino faleceram. Suas hipóteses sobre a sobrevivência provavelmente serão diferentes, o que resultará em um arquivo de submissão diferente. No entanto, assim como este arquivo, sua submissão deve conter:

Uma coluna "PassengerId" contendo os IDs de cada passageiro do arquivo test.csv.
Uma coluna "Survived" (que você criará!) com um "1" para as linhas em que você acredita que o passageiro sobreviveu e um "0" onde você prevê que o passageiro faleceu.

In [7]:
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# Explorando padrões

Inicialmente, vamos analisar o porque da submissão exemplo considerar que todos os homens morreram e todas as mulheres sobreviveram. Para isso, vamos checar as taxas de sobrevivência de todos os passageiros do arquivo train.csv

## Taxa de sobrevivencia feminina:

In [8]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% de mulheres que sobreviveram:", rate_women)

% de mulheres que sobreviveram: 0.7420382165605095


## Taxa de sobrevivencia masculina:

In [9]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% de homens que sobreviveram:", rate_men)

% de homens que sobreviveram: 0.18890814558058924


A partir disso, é possível observar que quase 75% das mulheres a bordo sobreviveram, enquanto apenas 19% dos homens conseguiram sobreviver. Visto que o gênero parece ser um indicador tão forte de sobrevivência, o arquivo de submissão em gender_submission.csv não é uma má primeira suposição!

No entanto, no final do dia, essa submissão baseada em gênero fundamenta suas previsões em apenas uma coluna. Como você pode imaginar, considerando múltiplas colunas, podemos descobrir padrões mais complexos que podem levar a previsões mais bem fundamentadas. Visto que é bastante difícil considerar várias colunas ao mesmo tempo (ou levaria muito tempo para analisar todos os possíveis padrões em muitas colunas diferentes simultaneamente), usaremos aprendizado de máquina para automatizar esse processo para nós.

# Criando um modelo Random Forest para o projeto

Vamos construir o que é conhecido como um modelo de floresta aleatória (random forest). Este modelo é composto por várias "árvores" que considerarão individualmente os dados de cada passageiro e votarão se o indivíduo sobreviveu ou não. Em seguida, o modelo de floresta aleatória toma uma decisão democrática: o resultado com a maioria dos votos vence!


A célula de código abaixo procura padrões em cinco colunas diferentes ("Pclass", "Sex", "SibSp" e "Parch", "Age") dos dados. Ela constrói as árvores no modelo de floresta aleatória com base nos padrões no arquivo train.csv antes de gerar previsões para os passageiros no arquivo test.csv. O código também salva essas novas previsões em um arquivo CSV chamado submission.csv.

In [22]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'name': test_data.Name, 'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [23]:
submission_data = pd.read_csv("submission.csv")
submission_data.head(20)

,name,PassengerId,Survived
0,"Kelly, Mr. James",892,0
1,"Wilkes, Mrs. James (Ellen Needs)",893,0
2,"Myles, Mr. Thomas Francis",894,0
3,"Wirz, Mr. Albert",895,0
4,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",896,1
5,"Svensson, Mr. Johan Cervin",897,0
6,"Connolly, Miss. Kate",898,1
7,"Caldwell, Mr. Albert Francis",899,0
8,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",900,1
9,"Davies, Mr. John Samuel",901,0


# Checando seu score localmente

Vamos usar os dados reais para ver a acurácia sem realizar a submissão no kaggle, apesar de ser recomendado para todos tentarem realizar sua primeira submissão e várias outras, tentando aumentar seu score

In [24]:
import re
real_data = pd.read_csv("titanic.csv")

# Para o DataFrame de dados reais (real_data)
for i, name in enumerate(real_data['name']):
    if '"' in name:
        real_data['name'][i] = re.sub('"', '', name)

# Para o DataFrame de previsões (submission_data)
for i, name in enumerate(submission_data['name']):
    if '"' in name:
        submission_data['name'][i] = re.sub('"', '', name)

survived = []

# Para o DataFrame de previsões (submission_data)
for name in submission_data['name']:
    survived.append(int(real_data.loc[real_data['name'] == name]['survived'].values[-1]))



<ipython-input-24-336da7db0be6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data['name'][i] = re.sub('"', '', name)
<ipython-input-24-336da7db0be6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_data['name'][i] = re.sub('"', '', name)


In [25]:
from sklearn.metrics import accuracy_score

# Calcula a acurácia
accuracy = accuracy_score(survived, submission_data['Survived'])

print(f"Acurácia: {accuracy * 100:.2f}%")


Acurácia: 78.01%


# Testando sua submissão no kaggle:

Certifique-se de que o seu notebook gere a mesma mensagem acima (Sua submissão foi salva com sucesso!) antes de prosseguir.

Faça o download desse arquivo no collab(caso esteja nele) e realize o upload no kaggle, na competição. Lá, você poderá checar o seu desempenho.

Certifique-se de que a opção "Salvar e Executar Tudo" esteja selecionada e, em seguida, clique no botão "Salvar".
Isso gera uma janela no canto inferior esquerdo do caderno. Depois que ela terminar de ser executada, clique no número à direita do botão "Salvar Versão". Isso abrirá uma lista de versões no lado direito da tela. Clique nos três pontos (...) à direita da versão mais recente e selecione "Abrir no Visualizador".
Clique na guia "Dados" na parte superior da tela. Em seguida, clique no botão "Enviar" para enviar seus resultados.